In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
df = pd.read_csv('Data/train_indessa.csv')

In [3]:
df.drop('batch_enrolled', axis=1, inplace=True)
df.drop('emp_title', axis=1, inplace=True)
df.drop('desc', axis=1, inplace=True)
df.drop('member_id', axis=1, inplace=True)

In [4]:
pd.set_option('display.max_columns', 45)

# Handling Missing Values

In [5]:
df['emp_length'].fillna('10+ years', inplace=True)

In [6]:
df['annual_inc'].fillna(6500, inplace=True)

In [7]:
df['title'].fillna('Debt consolidation', inplace=True)
df.dropna(axis='index', how='all', subset=['delinq_2yrs'], inplace=True)

In [8]:
df['revol_util'].fillna(24, inplace=True)
df['collections_12_mths_ex_med'].fillna(0.0, inplace=True)

In [9]:
def impute_mths_since_derog(cols):
    mths_since_derog = cols[0]
    delinq_2yrs = cols[1]
    
    if pd.isnull(mths_since_derog):
        if delinq_2yrs==0.0:
            return 52
        else:
            return 18
    
    else:
        return mths_since_derog

In [10]:
df['mths_since_last_major_derog'] = df[['mths_since_last_major_derog', 'delinq_2yrs']].apply(impute_mths_since_derog, axis=1)

In [11]:
df.drop('verification_status_joint', axis=1, inplace=True)

In [12]:
df['tot_coll_amt'].fillna(0.0, inplace=True)
df['tot_cur_bal'].fillna(0.0, inplace=True)
df['total_rev_hi_lim'].fillna(0.0, inplace=True)

In [13]:
df.drop('mths_since_last_record', axis=1, inplace=True)

In [14]:
df.drop('mths_since_last_delinq', axis=1, inplace=True)

# FEATURE ENGINEERING

In [15]:
def clean_term(term):
    ls = term.split(' ')
    return np.int64(ls[0])

df['term'] = df['term'].apply(clean_term)

In [16]:
def clean_emp_length(length):
    if length=='10+ years':
        x = length.split('+')
        return np.int64(x[0])
    elif length=='< 1 year':
        x = length.split(' ')
        return np.int64(x[1])
    else:
        x = length.split(' ')
        return np.int64(x[0])
    
df['emp_length'] = df['emp_length'].apply(clean_emp_length)

In [17]:
df.drop('title', axis=1, inplace=True)

In [18]:
def clean_zip(zip):
    x = zip.split('x')
    return np.int64(x[0])

df['zip_code'] = df['zip_code'].apply(clean_zip)

In [19]:
df.drop('addr_state', axis=1, inplace=True)

In [20]:
def clean_last_week_pay(week):
    x = week.split('th')
    if x[0] == 'NA':
        return 0
    else:
        return np.int64(x[0])

df['last_week_pay'] = df['last_week_pay'].apply(clean_last_week_pay)

In [21]:
df_clean = pd.get_dummies(df, drop_first=True)

In [22]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

In [23]:
X = df_clean.drop('loan_status', axis=1)
y = df_clean['loan_status']

In [24]:
X2D = pca.fit_transform(X)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier()

In [27]:
forest_clf.fit(X_train, y_train)

RandomForestClassifier()

In [28]:
y_pred = forest_clf.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[80917,   640],
       [15889,  9037]])

In [30]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.8447733441018754

# TEST DATA PREDICTION

In [31]:
df_test = pd.read_csv('Data/test_indessa.csv')

In [32]:
df_test.drop('batch_enrolled', axis=1, inplace=True)
df_test.drop('emp_title', axis=1, inplace=True)
df_test.drop('desc', axis=1, inplace=True)
df_test.drop('member_id', axis=1, inplace=True)
df_test.drop('verification_status_joint', axis=1, inplace=True)
df_test.drop('mths_since_last_record', axis=1, inplace=True)
df_test.drop('mths_since_last_delinq', axis=1, inplace=True)
df_test.drop('title', axis=1, inplace=True)
df_test.drop('addr_state', axis=1, inplace=True)

In [33]:
df_test['emp_length'].fillna('10+ years', inplace=True)
df_test['annual_inc'].fillna(6500, inplace=True)
df_test['revol_util'].fillna(24, inplace=True)
df_test['collections_12_mths_ex_med'].fillna(0.0, inplace=True)
df_test['mths_since_last_major_derog'] = df_test[['mths_since_last_major_derog', 'delinq_2yrs']].apply(impute_mths_since_derog, axis=1)

In [34]:
df_test['tot_coll_amt'].fillna(0.0, inplace=True)
df_test['tot_cur_bal'].fillna(0.0, inplace=True)
df_test['total_rev_hi_lim'].fillna(0.0, inplace=True)

In [35]:
df_test.describe()

,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,collections_12_mths_ex_med,mths_since_last_major_derog,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
count,354951.000000,354951.000000,354951.000000,354951.000000,3.549510e+05,354951.000000,354938.000000,354938.000000,354938.000000,354938.000000,3.549510e+05,354951.000000,354938.000000,354951.000000,354951.000000,354951.000000,354951.000000,354951.000000,354951.000000,354938.000000,3.549510e+05,3.549510e+05,3.549510e+05
mean,14751.767920,14738.287116,14698.770903,13.252396,7.502401e+04,18.184447,0.314432,0.694654,11.552781,0.195981,1.692005e+04,55.064623,25.269030,1756.867641,0.399299,46.221362,4.913062,0.014498,46.566966,0.004956,2.244993e+02,1.282178e+05,2.949783e+04
std,8437.019324,8431.045701,8443.341658,4.385250,6.393871e+04,25.174050,0.865533,1.000579,5.326112,0.579484,2.243219e+04,23.810774,11.836602,2098.587607,4.082242,409.763865,63.128236,0.135940,15.041033,0.075333,1.547464e+04,1.520161e+05,3.621487e+04
min,500.000000,500.000000,0.000000,5.320000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
25%,8000.000000,8000.000000,8000.000000,9.990000,4.500000e+04,11.890000,0.000000,0.000000,8.000000,0.000000,6.441000e+03,37.600000,17.000000,441.345000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.000000e+00,2.311100e+04,1.160000e+04
50%,13000.000000,13000.000000,13000.000000,12.990000,6.500000e+04,17.650000,0.000000,0.000000,11.000000,0.000000,1.187300e+04,56.000000,24.000000,1074.120000,0.000000,0.000000,0.000000,0.000000,52.000000,0.000000,0.000000e+00,6.510100e+04,2.180000e+04
75%,20000.000000,20000.000000,20000.000000,16.200000,9.000000e+04,23.940000,0.000000,1.000000,14.000000,0.000000,2.081100e+04,73.500000,32.000000,2243.075000,0.000000,0.000000,0.000000,0.000000,52.000000,0.000000,0.000000e+00,1.954315e+05,3.770000e+04
max,35000.000000,35000.000000,35000.000000,28.990000,9.000000e+06,9999.000000,39.000000,33.000000,76.000000,54.000000,2.904836e+06,182.800000,169.000000,23062.450000,286.747566,29282.070000,5569.920000,20.000000,188.000000,5.000000,9.152545e+06,4.447397e+06,9.999999e+06


In [36]:
df_test['delinq_2yrs'].fillna(0, inplace=True)
df_test['inq_last_6mths'].fillna(1, inplace=True)
df_test['open_acc'].fillna(6, inplace=True)
df_test['pub_rec'].fillna(0, inplace=True)
df_test['total_acc'].fillna(15, inplace=True)
df_test['acc_now_delinq'].fillna(0, inplace=True)

In [37]:
df_test['term'] = df_test['term'].apply(clean_term)
df_test['emp_length'] = df_test['emp_length'].apply(clean_emp_length)
df_test['zip_code'] = df_test['zip_code'].apply(clean_zip)
df_test['last_week_pay'] = df_test['last_week_pay'].apply(clean_last_week_pay)

In [38]:
df_test_clean = pd.get_dummies(df_test, drop_first=True)

In [39]:
pd.set_option('max_column', 90)

In [40]:
df_test_clean['home_ownership_mortage'] = 0

In [41]:
test_pred = forest_clf.predict(df_test_clean)

In [44]:
df_test_copy = pd.read_csv('Data/test_indessa.csv')

In [47]:
test_pred_proba = forest_clf.predict_proba(df_test_clean)

In [58]:
temp = pd.read_csv('Data/submission.csv')

In [67]:
temp.loc[temp['loan_status']==1, 'loan_status'] = 0.99
temp.loc[temp['loan_status']==0, 'loan_status'] = 0.01

In [68]:
temp.to_csv('Data/submission.csv', index=False)

In [69]:
temp.loc[temp['member_id']==1004221]

,member_id,loan_status
86662,1004221,0.01
